In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download -d puneet6060/intel-image-classification

intel-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip intel-image-classification.zip

Archive:  intel-image-classification.zip
replace seg_pred/seg_pred/10004.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import tensorflow as tf
import cv2 
import matplotlib.pyplot as plt
import os
import numpy as np 
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
import matplotlib.gridspec as gridspec
from random import randint
from keras.utils import to_categorical
from tensorflow.keras import layers, models

**Data Augmentation**

In [ ]:
def get_images(directory):
    images = []
    labels = []
    
    for dir in os.listdir(directory):
        label = -1
        if(dir == "buildings"):
            label = 0
        if(dir == "forest"):
            label = 1
        if(dir == "glacier"):
            label = 2
        if(dir == "mountain"):
            label = 3
        if(dir == "sea"):
            label = 4
        if(dir == "street"):
            label = 5
        for image_dir in os.listdir(directory +"//"+ dir):
            image = cv2.imread(directory +"//"+ dir + "//"+ image_dir)
            image = cv2.resize(image, (150,150))
            images.append(image)
            labels.append(label)
    return shuffle(images, labels)

In [ ]:
images, labels = get_images("/content/seg_train/seg_train")
images = np.array(images)
labels = np.array(labels)

In [ ]:
img_height=img_width=150
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [ ]:
images.shape

(14034, 150, 150, 3)

In [ ]:
labels = to_categorical(labels)
images = images.astype('float32')
images /= 255.0

In [ ]:
model = models.Sequential()
model.add(data_augmentation)
model.add(layers.Conv2D(filters = 64, kernel_size=(7,7), strides=(2,2), activation='relu', input_shape=(150,150,3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(layers.Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(layers.Conv2D(filters=128,kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(layers.Conv2D(filters=156, kernel_size=(2,2), strides=(1,1), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(72, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 150, 150, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 72, 72, 64)        9472      
                                                                 
 batch_normalization (BatchN  (None, 72, 72, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 35, 35, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 35, 35, 96)        55392     
                                                                 
 batch_normalization_1 (Batc  (None, 35, 35, 96)      

In [ ]:
trained = model.fit(images, labels, epochs =64)

Epoch 1/64
439/439 [==============================] - 27s 40ms/step - loss: 1.4956 - accuracy: 0.4249
Epoch 2/64
439/439 [==============================] - 16s 37ms/step - loss: 1.1768 - accuracy: 0.5545
Epoch 3/64
439/439 [==============================] - 17s 38ms/step - loss: 1.0756 - accuracy: 0.6059
Epoch 4/64
439/439 [==============================] - 16s 38ms/step - loss: 1.0034 - accuracy: 0.6365
Epoch 5/64
439/439 [==============================] - 16s 37ms/step - loss: 0.9267 - accuracy: 0.6728
Epoch 6/64
439/439 [==============================] - 16s 37ms/step - loss: 0.9104 - accuracy: 0.6845
Epoch 7/64
439/439 [==============================] - 16s 37ms/step - loss: 0.8599 - accuracy: 0.6985
Epoch 8/64
439/439 [==============================] - 17s 38ms/step - loss: 0.8317 - accuracy: 0.7115
Epoch 9/64
439/439 [==============================] - 16s 37ms/step - loss: 0.7851 - accuracy: 0.7306
Epoch 10/64
439/439 [==============================] - 17s 38ms/step - loss: 0.758

In [ ]:
test_images, test_labels = get_images('/content/seg_test/seg_test')

In [ ]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)
test_labels = to_categorical(test_labels)
test_images = test_images.astype('float32')
test_images /= 255.0

In [ ]:
pred_images, pred_labels = get_images('/content/seg_pred')
pred_images = np.array(pred_images)

In [ ]:
#Session crashed after using all RAM

In [ ]:
#from sklearn.metrics import classification_report
#print(classification_report(test_labels,pred_images))
#will give us the classification report. then we can tune hyperparameters like in previous assignment.